<a href="https://colab.research.google.com/github/Hemachandran-D45/vectordb/blob/main/RAG_Pipeline_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_community -q
!pip install langchain_huggingface -q
!pip install huggingface_hub -q
!pip install langchain_core -q

In [ ]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("/content/RohitSharma.txt")
loader.load()

In [3]:
rohit_doc = loader.load()

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

In [6]:
documents =text_splitter.split_documents(rohit_doc)

In [7]:
#for i, chunk in enumerate(documents):
 # print(f"Chunk {i+1} (Length: {len(chunk.page_content)} character)")
  #print(chunk.page_content)

In [8]:
!pip install chromadb -q

In [9]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
db = Chroma.from_documents(documents, HuggingFaceEmbeddings())

In [11]:
import os
from langchain.llms import HuggingFaceHub
from google.colab import userdata
sec_key = userdata.get("Rag_Pipeline")

In [30]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = sec_key
repo_id = "google/flan-t5-large"
task = "text2text-generation"



In [38]:
llm = HuggingFaceHub(repo_id=repo_id, task=task)

In [32]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the question based only on provided information.
{context}
Question: {input}""")

In [33]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)


In [34]:
retriever = db.as_retriever()

In [35]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [40]:
response=retrieval_chain.invoke({"input":"You won the T20 World Cup 2024"})

In [ ]:
response

In [41]:
response['answer']

'Rohit Sharma'